Import

In [1]:
import openai
from qdrant_client import QdrantClient
from transformers import AutoTokenizer, AutoModel
import torch

/home/cheetos/Bureau/Rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Client LlamaCpp

In [ ]:
openai_client = openai.OpenAI(
    base_url="http://localhost:8080/v1",  # URL de votre serveur llamacpp
    api_key="sk-no-key-required"
)

Fonction

In [ ]:

def load_embedding_model(model_name="intfloat/multilingual-e5-large-instruct"):
    """
    Charge le modèle d'embeddings et le tokenizer depuis Hugging Face.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)

    return tokenizer, model, device







def vectorize_query(query: str, tokenizer, model, device):
    """
    Vectorise une requête utilisateur pour le retriever.
    """
    query = f"query: {query}"

    inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        embedding = model(**inputs).last_hidden_state.mean(dim=1)

    return embedding.cpu().numpy()







def search_qdrant(query_vector, client, collection_name, top_k=3):
    """
    Effectue une recherche dans la collection Qdrant à partir d'un vecteur de requête.
    """
    query_vector = query_vector[0].tolist()

    results = client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=top_k,
        with_payload=True
    )

    return results







def retrieve_and_generate_answer(query: str, collection_name="GameRag"):
    """
    Recherche dans la base de données, puis génère une réponse via llamacpp.
    """

    tokenizer, model, device = load_embedding_model()

    qdrant_client = QdrantClient(url="http://localhost:6333")

    query_vector = vectorize_query(query, tokenizer, model, device)

    results = search_qdrant(query_vector, qdrant_client, collection_name)

    retrieved_texts = [result.payload.get("texte", "Texte non disponible") for result in results]

    context = "\n\n".join(retrieved_texts)
    prompt = f"Répond aux questions en te basant sur le contexte :\n\nContext :\n{context}\n\nQuestion :\n{query}"

    response = openai_client.completions.create(
        model="",
        temperature=0.7,
        top_p=0.9,
        prompt=prompt,
        stream=True
    )

    for chunk in response:
        print(chunk.choices[0].text, end='', flush=True)
    
    return prompt

Execution du script

In [6]:
# Exemple d'utilisation
query = "Comment puis-je améliorer mes performances en division ?"
a = retrieve_and_generate_answer(query)

/tmp/ipykernel_44419/1593052784.py:37: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


 "Pour améliorer vos performances en division, suivez ces recommandations :
1. **Pratiquez les tables de division** : Utilisez des cartes ou des applications pour mémoriser les tables de division jusqu'à 20. Cela vous aidera à effectuer des divisions plus rapidement et avec plus de précision.
2. **Entraînez-vous avec des calculs mentaux** : Utilisez des techniques comme la décomposition des nombres pour simplifier les divisions. Par exemple, au lieu de diviser 25 par 5, divisez 25 par 2 et multipliez le résultat par 2.
3. **Utilisez des outils** : Si vous avez des outils disponibles, comme des calculatrices, utilisez-les pour vérifier vos réponses et vous familiariser avec les processus de division.
4. **Faites des exercices chronométrés** : Entraînez-vous à effectuer des divisions en temps limité pour améliorer votre rapidité. Essayez de diviser des nombres simples (comme 20 par 4) en moins de 5 secondes.
5. **Analysez vos erreurs** : Tenez un journal de vos erreurs pour identifier le

In [8]:
a

'Contexte :\nRéponses aux questions d\'amélioration :\nQ: "Quelles sont vos recommandations pour améliorer mes performances ?"\nR: "Basé sur vos données, voici mes recommandations : concentrez-vous sur les divisions en particulier avec des nombres à 2 chiffres, pratiquez les calculs mentaux rapides pour réduire votre temps de réponse, essayez d\'atteindre le niveau 8 en moins de temps en optimisant vos déplacements, travaillez sur les exercices de puissances qui sont actuellement votre point faible."\nQ: "Comment puis-je réduire mon temps de réponse ?"\nR: "Pour réduire votre temps de réponse actuel de 6.5 secondes : utilisez des techniques de calcul mental comme la décomposition des nombres, pratiquez les tables de multiplication jusqu\'à 12, faites des sessions d\'entraînement chronométrées. Votre objectif devrait être d\'atteindre 5 secondes par réponse."\n\n\n\nRéponses aux questions de stratégie :\nQ: "Quelle stratégie devrais-je adopter pour améliorer mon score ?"\nR: "Visez 5 se